<a href="https://colab.research.google.com/github/estherndegwa/MACHINE-LEARNING-GROUP-TASKS/blob/main/GROUP_5_ML_TASK_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 3.7 MB/s 


In [ ]:
!pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification,make_moons
from category_encoders import BackwardDifferenceEncoder
from matplotlib.colors import ListedColormap
from keras.optimizers import adam_v2
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report,confusion_matrix

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ImportError: ignored

GETTING THE DATASET AND PREPARING IT

In [ ]:
data=pd.read_csv('/content/Housing Pricing.csv')

FileNotFoundError: ignored

In [ ]:
data.head(2)

In [ ]:
data.shape

In [ ]:
data.drop('Id',inplace=True,axis=1)

In [ ]:
data.head(1)

In [ ]:
original_features=list(data.columns)
len(original_features)

In [ ]:
features_missing_values=list(data.columns[data.isna().any()])
len(features_missing_values)

In [ ]:
len(data)

FILL IN THE MISSING VALUES

In [ ]:
bfill,mean_fill=[],[]

In [ ]:
for feature in features_missing_values:
  print(feature,' ',data[feature].isna().sum()/len(data)*100,' ',data[feature].dtype)
  if data[feature].dtype=='float64':
    mean_fill.append(feature)
  else:
    bfill.append(feature)

In [ ]:
mean_fill

In [ ]:
for feature in mean_fill:
    data[feature].fillna(data[feature].mean(),inplace=True)

In [ ]:
for feature in bfill:
    data[feature].fillna(method='bfill',inplace=True)

In [ ]:
data.columns[data.isna().any()]

In [ ]:
data['FireplaceQu'].isna().sum()

In [ ]:
data['FireplaceQu'].fillna(method='ffill',inplace=True)

**ENCODE THE DATASET**



In [ ]:
categorical_features=list(data.select_dtypes(include=['object']).columns)
categorical_features

In [ ]:
numerical=list(set(original_features)-set(categorical_features))
numerical

In [ ]:
categorical_features

In [ ]:
nominal=['MSZoning','LandContour','Neighborhood']
ordinal=list(set(categorical_features)-set(nominal))

In [ ]:
target=['SalePrice']

In [ ]:
df_nominal=pd.get_dummies(data[nominal])

In [ ]:
for feature in ordinal:
  data[feature]=data[feature].astype('category').cat.codes

df_ordinal=data[ordinal]

In [ ]:
data[numerical]

In [ ]:
new_data=pd.concat([df_nominal,df_ordinal,data[numerical]],axis=1)

In [ ]:
new_data.shape

**STANDARDIZE THE DATASET**

In [ ]:
new_data.drop('SalePrice',axis=1,inplace=True)

In [ ]:
X=new_data.to_numpy()

In [ ]:
X[0]

In [ ]:
X=StandardScaler().fit_transform(X)

In [ ]:
X[0]

In [ ]:
y=data[target]

In [ ]:
y=y.to_numpy()

In [ ]:
y.shape

In [ ]:
y

In [ ]:
y=StandardScaler().fit_transform(y)

In [ ]:
y

In [ ]:
X.shape

**FEATURE SELETION USING L1**

In [ ]:
numerics = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

In [ ]:
X = data.drop("SalePrice",1)
y = data["SalePrice"]
regressor=LassoCV()
regressor.fit(X,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

In [ ]:
new_x = PowerTransformer().fit_transform(data)

test_data = pd.DataFrame(new_x,columns=data.columns)
test_data = pd.concat([test_data[numerical],test_data[target]],axis=1)
test_data

In [ ]:
coef_col = pd.Series(regressor.coef_,index=X.columns)

#print(coef_col)
sorted_coef = coef_col.sort_values()
matplotlib.rcParams['figure.figsize']=(10.0,7.0)
sorted_coef.plot(kind="barh")

no_of_selected_features = sum(coef_col != 0)
no_of_rejected_features = sum(coef_col == 0)
total_features = no_of_selected_features + no_of_rejected_features
print("L1 selected only ",no_of_selected_features," features out of ",total_features," from the transformed dataset ")
plt.title("Feature Selection Using L1 Embedder")

**TRAINING AND TESTING THE MODEL**

In [ ]:
def plot_data(X, y, figsize=None):
    if not figsize:
        figsize = (8, 6)
    plt.figure(figsize=figsize)
    plt.plot(X[y==0, 0], X[y==0, 1], 'or', label=0)
    plt.plot(X[y==1, 0], X[y==1, 1], 'ob', label=1)
    plt.xlim((min(X[:, 0])-0.1, max(X[:, 0])+0.1))
    plt.ylim((min(X[:, 1])-0.1, max(X[:, 1])+0.1))
    plt.xlabel('x_1')
    plt.ylabel('x_2')
    plt.legend()

In [ ]:
def plot_decision_boundary(func, X, y,epochs, figsize=(9, 6)):

    xmin, ymin = X.min(axis=1) - 0.1
    xmax, ymax = X.max(axis=1) + 0.1
    hticks = np.linspace(xmin, xmax, epochs+1)
    vticks = np.linspace(ymin, ymax, epochs+1)
    
    xx, yy = np.meshgrid(hticks, vticks)
    xy = np.c_[xx.ravel(), yy.ravel()]
    z = func(xy)
    zz = z.reshape(xx.shape)

    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    
    fig, ax = plt.subplots(figsize=figsize)
    contour = plt.contourf(xx, yy, zz, cmap=cm, alpha=0.8)
    
    ax_z = fig.colorbar(contour)
    ax_z.set_label("$P(y = 1)$")
    ax_z.set_ticks([0, 0.2, 0.5, 0.7, 1])
    
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright)
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    plt.xlabel("x_1")
    plt.ylabel("x_2")

    plot_data(X,y)

In [ ]:
model=Sequential()
model.add(Dense(1, input_shape=(1,), activation='sigmoid'))

model.compile('adam', 'binary_crossentropy', metrics=['Precision','Recall'])

history = model.fit(X, y, verbose=0, epochs=100)

In [ ]:
epochs=200

model_2 = Sequential()
model_2.add(Dense(4, input_shape=(2,), activation='tanh'))
model_2.add(Dense(2, activation='tanh'))
model_2.add(Dense(1, activation='sigmoid'))

model_2.compile(Adam(learning_rate=0.01), 'binary_crossentropy', metrics=['Precision','Recall'])

history = model_2.fit(X, y, verbose=0, epochs=epochs)


In [ ]:
model_2.summary()

In [ ]:
historydf=pd.DataFrame(history.history, index=history.epoch)
historydf.head()

In [ ]:
historydf['F-Measure']=(2*historydf['precision']*historydf['recall'])/(historydf['precision']+historydf['recall'])

In [ ]:
historydf.head()

In [ ]:
historydf.drop(['precision','recall'],axis=1,inplace=True)

In [ ]:
final_loss=historydf['loss'].to_numpy()[-1]
final_f1=historydf['F-Measure'].to_numpy()[-1]

In [ ]:
historydf.plot()
plt.title('Final Loss: %.3f, Final F1_score: %.3f' % (final_loss, final_f1))

In [ ]:
plt.show()

In [ ]:
plot_decision_boundary(lambda X_test: model_2.predict(X_test), X_test, y_test,epochs)